📄 File Notes

    This script reads in the systemdata.yaml file located in the same directory as the notebook.

    It uses the interim_data_dir and compressed_data_dir paths specified in that YAML file.

    It then runs the same compression logic used by multiCam_RT_videoAcquisition_v5.py.

    Specifically, it searches for .avi files inside interim_data_dir, compresses them into .mp4 files using ffmpeg, and saves them to the corresponding folder inside compressed_data_dir.

    Important: Unlike the compression button in multiCam_RT_videoAcquisition_v5.py, this notebook does not immediately delete the .avi files. Instead, it compresses and moves them first so you can manually verify the .mp4 output.

    Once you're satisfied with the .mp4 files, simply run the final cell in this notebook to delete the corresponding .avi files from interim_data_dir.

# data_aq Kernal Required

In [3]:
# # Reset notebook kernel to pull in new videos
# import IPython
# app = IPython.get_ipython()
# app.kernel.do_shutdown(restart=True)  # restarts the kernel

In [4]:
import os
import glob

import shutil
import subprocess
from pathlib import PurePath
import cv2
import multiCam_DLC_utils_v2 as clara  # Must be in your path
import gc
import time
from tqdm import tqdm


In [5]:
# ⚙️ Setup: Load user config and paths
user_cfg = clara.read_config()
read_dir = user_cfg['interim_data_dir']
write_dir = user_cfg['compressed_data_dir']
unit_ref = user_cfg['unitRef']
final_dir = user_cfg['final_data_dir']
print(f"Loaded paths for unit: {unit_ref}")
print('')
print(f'read_dir: {read_dir}')
print(f'write_dir: {write_dir}')
print(f'unit_ref: {unit_ref}')
print(f'final_dir: {final_dir}')

Loaded paths for unit: christielab

read_dir: C:\RawDataLocal
write_dir: C:\compressedData
unit_ref: christielab
final_dir: Z:\PHYS\ChristieLab\Lab Management\Grant\reachingData\compressed_data


In [6]:
# 📁 Discover all AVI videos to compress

def get_video_dirs():
    dirlist, destlist = [], []
    prev_date_list = [name for name in os.listdir(read_dir)]
    
    for date in prev_date_list:
        src_dir = os.path.join(read_dir, date, unit_ref)
        dest_dir = os.path.join(write_dir, date, unit_ref)
        if os.path.exists(src_dir):
            for sess in os.listdir(src_dir):
                dirlist.append(os.path.join(src_dir, sess))
                destlist.append(os.path.join(dest_dir, sess))
    
    return dirlist, destlist

dirlist, destlist = get_video_dirs()
print(f"Found {len(dirlist)} directories with videos.")
print(f'Verify raw files: {dirlist}')
print(f'Verify Compression Destination: {destlist}')


Found 257 directories with videos.
Verify raw files: ['C:\\RawDataLocal\\20241204\\christielab\\session001', 'C:\\RawDataLocal\\20241204\\christielab\\session002', 'C:\\RawDataLocal\\20241204\\christielab\\session003', 'C:\\RawDataLocal\\20241204\\christielab\\session004', 'C:\\RawDataLocal\\20241204\\christielab\\session005', 'C:\\RawDataLocal\\20241204\\christielab\\session006', 'C:\\RawDataLocal\\20250115\\christielab\\session001', 'C:\\RawDataLocal\\20250115\\christielab\\session002', 'C:\\RawDataLocal\\20250115\\christielab\\session003', 'C:\\RawDataLocal\\20250115\\christielab\\session004', 'C:\\RawDataLocal\\20250115\\christielab\\session005', 'C:\\RawDataLocal\\20250409\\christielab\\session001', 'C:\\RawDataLocal\\20250409\\christielab\\session002', 'C:\\RawDataLocal\\20250409\\christielab\\session003', 'C:\\RawDataLocal\\20250410\\christielab\\session001', 'C:\\RawDataLocal\\20250414\\christielab\\session001', 'C:\\RawDataLocal\\20250415\\christielab\\session001', 'C:\\RawDat

In [7]:
# 🧪 Function to compare frame counts

def test_vids(v, dest_path):
    try:
        vid_a = cv2.VideoCapture(v)
        frames_a = int(vid_a.get(cv2.CAP_PROP_FRAME_COUNT))
        vid_b = cv2.VideoCapture(dest_path)
        frames_b = int(vid_b.get(cv2.CAP_PROP_FRAME_COUNT))
        return (frames_a == frames_b) and (frames_a > 0)
    except:
        return False



In [8]:
# new test
def test_vids(v, dest_path):
    try:
        vid_a = cv2.VideoCapture(v)
        vid_b = cv2.VideoCapture(dest_path)
        frames_a = int(vid_a.get(cv2.CAP_PROP_FRAME_COUNT))
        frames_b = int(vid_b.get(cv2.CAP_PROP_FRAME_COUNT))
        vid_a.release()
        vid_b.release()
        return (frames_a == frames_b) and (frames_a > 0)
    except:
        return False

In [9]:
# 🗜️ Compress and replace .avi with .mp4 using ffmpeg

def compress_videos_in_dir(src_dir, dest_dir):
    


    # old code
    # avi_list = glob.glob(os.path.join(src_dir, '*.avi'))


    # New Code
    cam_patterns = [
        "*_frontCam-*.avi",
        "*_stimCam-*.avi",
        "*_sideCam-*.avi",
    ]
    avi_list = []
    for pat in cam_patterns:
        avi_list.extend(sorted(glob.glob(os.path.join(src_dir, pat))))
    # include any other AVIs not matching the three cams (keeps backward compatibility)
    remaining = set(glob.glob(os.path.join(src_dir, '*.avi'))) - set(avi_list)
    avi_list.extend(sorted(remaining))
    # New Code

    if not avi_list:
        return

    os.makedirs(dest_dir, exist_ok=True)
    processes = []

    for v in avi_list:
        vid_name = PurePath(v)
        dest_path = os.path.join(dest_dir, vid_name.stem + '.mp4')

        if not test_vids(v, dest_path):  # Don't recompress if valid
            env = os.environ.copy()
            env["PATH"] = r"C:\ffmpeg\bin;" + env["PATH"]
            cmd = f'ffmpeg -y -i "{v}" -c:v libx264 -preset veryfast -vf format=yuv420p -c:a copy -crf 17 -loglevel quiet "{dest_path}"'
            print(f"Compressing: {vid_name.name} ➜ {vid_name.stem}.mp4 ")
            processes.append(subprocess.Popen(cmd, env=env, shell=True))

    # Wait for all compressions to finish
    for p in processes:
        p.wait()

    # Validate and remove successful .avi
    for v in avi_list:
        vid_name = PurePath(v)
        dest_path = os.path.join(dest_dir, vid_name.stem + '.mp4')
        if test_vids(v, dest_path):
            #os.remove(v)
            print(f"✅ Compressed: {vid_name.name}")
        else:
            print(f"❌ Compression failed: {vid_name.name}")

            # 🧹 Copy non-avi files (e.g., YAML metadata)
    
    # ✅ Cleanup: Clear system-level file locks
    import gc
    gc.collect()
    print('Cleared videos from temporary memory')

def copy_metadata(src_dir, dest_dir):
    os.makedirs(dest_dir, exist_ok=True)
    metafiles = glob.glob(os.path.join(src_dir, '*'))
    for m in metafiles:
        if '.avi' not in m:
            mname = PurePath(m).name
            mdest = os.path.join(dest_dir, mname)
            if not os.path.isfile(mdest):
                shutil.copyfile(m, mdest)



In [10]:
import os
import glob

def estimate_compression_all_sessions(base_dir):
    session_dirs = glob.glob(os.path.join(base_dir, '*', '*', 'session*'))  # e.g. RawData/20250415/christielab/session001
    total_size = 0
    total_files = 0

    print("📦 Estimated compression job across all sessions:\n")

    for session in session_dirs:
        avi_files = glob.glob(os.path.join(session, '*.avi'))
        if not avi_files:
            continue

        session_size = 0
        print(f"📁 Session: {session}")
        for avi in avi_files:
            size_mb = os.path.getsize(avi) / (1024 * 1024)
            size_gb = size_mb /1000
            session_size += size_mb
            print(f"  • {os.path.basename(avi)} — {size_mb:.2f} MB --> {size_gb:.2F} GB")

        est_time = len(avi_files) * 8  # ~8s per video
        print(f"  🧮 Session total: {session_size:.2f} MB")
        print(f"  ⏱️  Estimated time: ~{est_time:.1f} seconds ({est_time/60:.1f} min)\n")

        # Actual Historical Compression time for 467.54 GB // 13 minutes
        compress_time_min = 13 
        file_size_total = 467.5
        gb_compressed_per_min =  file_size_total/ compress_time_min 
        gb_compressed_every_10_min = gb_compressed_per_min * 10
        print(f'Compression Speed: (( {gb_compressed_per_min:.0f} gb )) per 1 Minute')
        print(f'Compression Speed: (( {gb_compressed_every_10_min:.0f} gb )) per 10 Minute')

        total_size += session_size
        total_size_gb = total_size /1000
        est_compress_time = total_size_gb
        total_files += len(avi_files)

    
    estimated_compression_time_min = (total_size_gb /gb_compressed_per_min )
    estimated_compression_time_min
    print("🔚 Final summary:")
    print(f"🔢 Total files: {total_files}")
    print(f"📂 Total size: {total_size:.2f} MB") 
    print(f"📂 Total size: {total_size_gb:.2f} GB") 
    print(f"⏱️  Estimated total Compression time: ({estimated_compression_time_min:.1f} minutes)")
    return total_size_gb, estimated_compression_time_min

# Example usage:
total_size_gb, estimated_compression_time_min = estimate_compression_all_sessions(read_dir)


📦 Estimated compression job across all sessions:

📁 Session: C:\RawDataLocal\20250916\christielab\session002
  • 20250916_christielab_session002_sideCam-0000.avi — 8120.49 MB --> 8.12 GB
  • 20250916_christielab_session002_stimCam-0000.avi — 8072.59 MB --> 8.07 GB
  🧮 Session total: 16193.09 MB
  ⏱️  Estimated time: ~16.0 seconds (0.3 min)

Compression Speed: (( 36 gb )) per 1 Minute
Compression Speed: (( 360 gb )) per 10 Minute
📁 Session: C:\RawDataLocal\20250925\christielab\session001
  • 20250925_christielab_session001_frontCam-0000.avi — 9075.40 MB --> 9.08 GB
  • 20250925_christielab_session001_sideCam-0000.avi — 9075.40 MB --> 9.08 GB
  🧮 Session total: 18150.80 MB
  ⏱️  Estimated time: ~16.0 seconds (0.3 min)

Compression Speed: (( 36 gb )) per 1 Minute
Compression Speed: (( 360 gb )) per 10 Minute
📁 Session: C:\RawDataLocal\20250925\christielab\session002
  • 20250925_christielab_session002_frontCam-0000.avi — 46378.62 MB --> 46.38 GB
  • 20250925_christielab_session002_sideCam

### Pervious actual compression time

In [11]:
# 13 minutes for 467.54 GB
compress_time_min = 13 
file_size_total = 467.5

gb_compressed_per_min =  file_size_total/ compress_time_min 
gb_compressed_every_10_min = gb_compressed_per_min * 10
print(f'Compression Speed: (( {gb_compressed_per_min:.0f} gb )) per 1 Minute')
print(f'Compression Speed: (( {gb_compressed_every_10_min:.0f} gb )) per 10 Minute')


Compression Speed: (( 36 gb )) per 1 Minute
Compression Speed: (( 360 gb )) per 10 Minute


### This is the compression step
- stop here if you do not want to compress files

In [12]:
compression_fail_safe = False

In [13]:
# Usage
print(f'\n')
print(f"📂 Total size: {total_size_gb:.2f} GB") 
print(f"⏱️ Estimated total Compression time: ({estimated_compression_time_min:.1f} minutes)") 
print(f'\n\n')
if compression_fail_safe:
    print('⚠️Fail safe activated⚠️')
    print('Are you sure you want to begind compressing the raw .avi files into .mp4 files')
    print(f'The compressed .mp4 files will be placed here --> {write_dir}')
    print(' ')
    print('✅ if Yes, simply run this cell again and it will delete all .avi files that have a .mp4 copy')
    print('❌ if No, do not re-run this cell')
    gc.collect()
    time.sleep(1)  # Let the OS finish background cleanup
    compression_fail_safe = False
else:
    for src_dir, dest_dir in zip(dirlist, destlist):
        compress_videos_in_dir(src_dir, dest_dir)
        copy_metadata(src_dir, dest_dir)
    # Clear files from background
    gc.collect()
    time.sleep(1)  # Let the OS finish background cleanup
    print("\n🎉 All video compression completed.")
    print('Files cleared from background memory')




📂 Total size: 337.75 GB
⏱️ Estimated total Compression time: (9.4 minutes)



✅ Compressed: 20250916_christielab_session002_stimCam-0000.avi
✅ Compressed: 20250916_christielab_session002_sideCam-0000.avi
Cleared videos from temporary memory
✅ Compressed: 20250925_christielab_session001_frontCam-0000.avi
✅ Compressed: 20250925_christielab_session001_sideCam-0000.avi
Cleared videos from temporary memory
✅ Compressed: 20250925_christielab_session002_frontCam-0000.avi
✅ Compressed: 20250925_christielab_session002_sideCam-0000.avi
Cleared videos from temporary memory
✅ Compressed: 20250925_christielab_session003_frontCam-0000.avi
✅ Compressed: 20250925_christielab_session003_sideCam-0000.avi
Cleared videos from temporary memory
✅ Compressed: 20250926_christielab_session006_frontCam-0000.avi
✅ Compressed: 20250926_christielab_session006_sideCam-0000.avi
Cleared videos from temporary memory
✅ Compressed: 20250927_christielab_session001_frontCam-0000.avi
✅ Compressed: 20250927_christielab_se

In [14]:
import os
import glob
from pathlib import PurePath

def compare_raw_vs_compressed(raw_base_dir, comp_base_dir, unit='christielab'):
    raw_sessions = glob.glob(os.path.join(raw_base_dir, '*', unit, 'session*'))
    results = []

    for raw_session in raw_sessions:
        comp_session = raw_session.replace(raw_base_dir, comp_base_dir)
        raw_avi_files = glob.glob(os.path.join(raw_session, '*.avi'))
        comp_mp4_files = glob.glob(os.path.join(comp_session, '*.mp4'))
        comp_mp4_names = {PurePath(f).stem for f in comp_mp4_files}

        for avi in raw_avi_files:
            avi_name = PurePath(avi).stem
            has_match = avi_name in comp_mp4_names
            results.append((avi_name, has_match, raw_session))

    print("🎞️ Comparison of .avi and .mp4 files across sessions:")
    for name, match, session in results:
        status = "✅ MATCHED" if match else "❌ MISSING .mp4"
        
        print(f"{status}: {name}  ({session})")

    return results  # Optional: for inspection

def list_avi_to_delete(results):
    to_delete = []

    for name, matched, session in results:
        if matched:
            avi_path = os.path.join(session, name + '.avi')
            if os.path.exists(avi_path):
                to_delete.append(avi_path)

    print(f"\n🧹 {len(to_delete)} .avi files are safe to delete:\n")
    for path in to_delete:
        print(f"🗑️ {path}")
    
    return to_delete  # So you can manually delete or loop over them
results = compare_raw_vs_compressed(read_dir, write_dir)
avi_files_to_delete = list_avi_to_delete(results)

# After compressing or verifying videos
gc.collect()
time.sleep(1)  # Let the OS finish background cleanup

🎞️ Comparison of .avi and .mp4 files across sessions:
✅ MATCHED: 20250916_christielab_session002_sideCam-0000  (C:\RawDataLocal\20250916\christielab\session002)
✅ MATCHED: 20250916_christielab_session002_stimCam-0000  (C:\RawDataLocal\20250916\christielab\session002)
✅ MATCHED: 20250925_christielab_session001_frontCam-0000  (C:\RawDataLocal\20250925\christielab\session001)
✅ MATCHED: 20250925_christielab_session001_sideCam-0000  (C:\RawDataLocal\20250925\christielab\session001)
✅ MATCHED: 20250925_christielab_session002_frontCam-0000  (C:\RawDataLocal\20250925\christielab\session002)
✅ MATCHED: 20250925_christielab_session002_sideCam-0000  (C:\RawDataLocal\20250925\christielab\session002)
✅ MATCHED: 20250925_christielab_session003_frontCam-0000  (C:\RawDataLocal\20250925\christielab\session003)
✅ MATCHED: 20250925_christielab_session003_sideCam-0000  (C:\RawDataLocal\20250925\christielab\session003)
✅ MATCHED: 20250926_christielab_session006_frontCam-0000  (C:\RawDataLocal\20250926\chr

## moves final compressed files into Z-drive
- this step may not be needed by you if your already working with the Z-drive
- but in my case i am compressing the videos directly on my local SSD
- this step checks the final_dest path which you now set in the systemdata.yaml file, which points to my Z-drive, and copies all new sessions there

In [15]:
# 1 minutes for 467.54 GB
transfer_time_min = 1 
file_size_total = 281.39

gb_transfer_per_min =  file_size_total/ transfer_time_min 
gb_transfer_every_10_min = gb_transfer_per_min * 10
print(f'transfer Speed: (( {gb_transfer_per_min:.0f} gb )) per 1 Minute')
print(f'transfer Speed: (( {gb_transfer_every_10_min:.0f} gb )) per 10 Minute')


# Example usage
estimated_transfer_time_min = (total_size_gb /gb_transfer_per_min )

print(f'Estimated Transfer Time: {estimated_transfer_time_min:.0f} Minute')

transfer Speed: (( 281 gb )) per 1 Minute
transfer Speed: (( 2814 gb )) per 10 Minute
Estimated Transfer Time: 1 Minute


In [ ]:
def move_compressed_sessions(write_dir, final_dest,view_skipped=False):
    session_dirs = glob.glob(os.path.join(write_dir, '*', '*', 'session*'))
    print(f"📦 Preparing to copy {len(session_dirs)} session folders...\n")

    skip_counter = 0
    backup_counter = 0
    for session_path in session_dirs:
        rel_path = os.path.relpath(session_path, write_dir)  # keeps subfolder structure
        dest_path = os.path.join(final_dest, rel_path)

        if os.path.exists(dest_path):
            if view_skipped:
                skip_counter += 1
                print(f"⚠️  Skipping existing: {dest_path}") # set view_skipped=True if you want to look as the specfic files its passing over (because they are already backed up) 
            skip_counter += 1
            continue

        os.makedirs(os.path.dirname(dest_path), exist_ok=True)
        print(f"📁 Copying: {session_path} ➜ {dest_path}")
        shutil.copytree(session_path, dest_path)
        backup_counter += 1
    print('=======================================================================')
    print(f'⚠️{skip_counter} files were skipped. because they allready had backups')
    print('')
    print("\n✅ All eligible session folders have been copied!")
    print(f'📂 {backup_counter} session(s) were copied')


estimated_transfer_time_min = (total_size_gb /gb_transfer_per_min )
print(f'⏱️ Estimated Transfer Time: {estimated_transfer_time_min:.0f} Minute(s)\n\n')

move_compressed_sessions(write_dir, final_dir,view_skipped=False)


⏱️ Estimated Transfer Time: 1 Minute(s)


📦 Preparing to copy 376 session folders...



In [ ]:
print('\a')
import winsound
winsound.Beep(1000, 2000)  # 1000 Hz for 0.5 seconds




# STOP --> read before running cell bellow
- delete_matched_avi_files_with_progress() will delete the raw .avi files in the interim_data_dir
- you should manually check that the .mp4 files exist in the compressed_data_dir, and are not corrupted before running this cell
- this step has a fail safe built in so you do not accidently delete your raw files. in order for delete_matched_avi_files_with_progress() to delete your videos, the cell has to be run two times in a row. the second run turns off the fail safe

In [ ]:
# Do not change, unless you want to turn of the .avi deletion fail safe
# this fail_safe stops the cell bellow from deleting the .avi videos until you run it twice
fail_safe = True

In [ ]:
def delete_matched_avi_files(results, delete_avi_videos=False):
    if delete_avi_videos:
        matched_files = [(name, session) for name, matched, session in results if matched]
        deleted = []

        print(f"🔍 Attempting to delete {len(matched_files)} matched .avi files...\n")

        for name, session in tqdm(matched_files, desc="🧹 Deleting", unit="file"):
            avi_path = os.path.join(session, name + '.avi')
            if os.path.exists(avi_path):
                try:
                    start = time.time()
                    os.remove(avi_path)

                    # New Code: wait up to 120s for confirmation
                    timeout = 120
                    while os.path.exists(avi_path) and (time.time() - start) < timeout:
                        time.sleep(1)

                    if os.path.exists(avi_path):
                        print(f"⏭️ Skipped (not deleted after {timeout}s): {avi_path}")
                        continue

                    elapsed = time.time() - start
                    print(f"✅ Deleted: {avi_path} ({elapsed:.2f} s)")
                    deleted.append(avi_path)

                except Exception as e:
                    import traceback
                    print(f"❌ Error deleting {avi_path}: {e}")
                    traceback.print_exc()

        print(f"\n🧼 Done! Deleted {len(deleted)} files.")
        return deleted
    else:
        print('⚠️Fail safe activated⚠️')
        print('Are you sure you want to delete the raw .avi files?')
        print(' ')
        print('✅ if Yes, simply run this cell again and it will delete all .avi files that have a .mp4 copy')
        print('❌ if No, do not re-run this cell')

# Usage
if fail_safe:
    #results = compare_raw_vs_compressed(read_dir, write_dir)
    delete_matched_avi_files(results)
    fail_safe = False
else:
    results = compare_raw_vs_compressed(read_dir, write_dir)
    delete_matched_avi_files(results,delete_avi_videos = True)

# Commpletion Sound
print('\a')
import winsound
winsound.Beep(1000, 500)  # 1000 Hz for 0.5 seconds

🎞️ Comparison of .avi and .mp4 files across sessions:
✅ MATCHED: 20250916_christielab_session002_sideCam-0000  (C:\RawDataLocal\20250916\christielab\session002)
✅ MATCHED: 20250916_christielab_session002_stimCam-0000  (C:\RawDataLocal\20250916\christielab\session002)
✅ MATCHED: 20250925_christielab_session001_frontCam-0000  (C:\RawDataLocal\20250925\christielab\session001)
✅ MATCHED: 20250925_christielab_session001_sideCam-0000  (C:\RawDataLocal\20250925\christielab\session001)
✅ MATCHED: 20250925_christielab_session002_frontCam-0000  (C:\RawDataLocal\20250925\christielab\session002)
✅ MATCHED: 20250925_christielab_session002_sideCam-0000  (C:\RawDataLocal\20250925\christielab\session002)
✅ MATCHED: 20250925_christielab_session003_frontCam-0000  (C:\RawDataLocal\20250925\christielab\session003)
✅ MATCHED: 20250925_christielab_session003_sideCam-0000  (C:\RawDataLocal\20250925\christielab\session003)
✅ MATCHED: 20250926_christielab_session006_frontCam-0000  (C:\RawDataLocal\20250926\chr

🧹 Deleting:   0%|          | 0/24 [00:00<?, ?file/s]

In [ ]:
fail_safe = True

In [ ]:

def delete_matched_avi_files(results,delete_avi_videos=False):
    if delete_avi_videos:
        matched_files = [(name, session) for name, matched, session in results if matched]
        deleted = []

        print(f"🔍 Attempting to delete {len(matched_files)} matched .avi files...\n")

        for name, session in tqdm(matched_files, desc="🧹 Deleting", unit="file"):
            avi_path = os.path.join(session, name + '.avi')
            if os.path.exists(avi_path):
                try:
                    start = time.time()
                    os.remove(avi_path)
                    elapsed = time.time() - start
                    print(f"✅ Deleted: {avi_path} ({elapsed:.2f} s)")
                    deleted.append(avi_path)
                except Exception as e:
                    import traceback
                    print(f"❌ Error deleting {avi_path}: {e}")
                    traceback.print_exc()


        print(f"\n🧼 Done! Deleted {len(deleted)} files.")
        return deleted
    else:
        print('⚠️Fail safe activated⚠️')
        print('Are you sure you want to delete the raw .avi files?')
        print(' ')
        print('✅ if Yes, simply run this cell again and it will delete all .avi files that have a .mp4 copy')
        print('❌ if No, do not re-run this cell')

# Usage
if fail_safe:
    #results = compare_raw_vs_compressed(read_dir, write_dir)
    delete_matched_avi_files(results)
    fail_safe = False
else:
    results = compare_raw_vs_compressed(read_dir, write_dir)
    delete_matched_avi_files(results,delete_avi_videos = True)

# Commpletion Sound
print('\a')
import winsound
winsound.Beep(1000, 500)  # 1000 Hz for 0.5 seconds


🎞️ Comparison of .avi and .mp4 files across sessions:
✅ MATCHED: 20250915_christielab_session020_stimCam-0000  (C:\RawDataLocal\20250915\christielab\session020)
✅ MATCHED: 20250916_christielab_session002_frontCam-0000  (C:\RawDataLocal\20250916\christielab\session002)
✅ MATCHED: 20250916_christielab_session002_sideCam-0000  (C:\RawDataLocal\20250916\christielab\session002)
✅ MATCHED: 20250916_christielab_session002_stimCam-0000  (C:\RawDataLocal\20250916\christielab\session002)
✅ MATCHED: 20250918_christielab_session001_frontCam-0000  (C:\RawDataLocal\20250918\christielab\session001)
✅ MATCHED: 20250918_christielab_session001_sideCam-0000  (C:\RawDataLocal\20250918\christielab\session001)
✅ MATCHED: 20250918_christielab_session001_stimCam-0000  (C:\RawDataLocal\20250918\christielab\session001)
✅ MATCHED: 20250918_christielab_session002_frontCam-0000  (C:\RawDataLocal\20250918\christielab\session002)
✅ MATCHED: 20250918_christielab_session002_sideCam-0000  (C:\RawDataLocal\20250918\chri

🧹 Deleting:   0%|          | 0/19 [00:00<?, ?file/s]

## Completetion Bell

In [ ]:
print('\a')
import winsound
winsound.Beep(1000, 500)  # 1000 Hz for 0.5 seconds





# BACKUP Deletion Instructions
### If the above cell get stuck and cannot delete a specific file follow these steps
1. First try restarting your computer and re-running this script.
2. If that does not work, you can manually delete it using this poweshell command code in admin mode

### Option #1
- uncomment the cell bellow
- replace paths bellow with locked file
- run cell

### option #2 (Admin)
-  "windows key" + R 
- Type: cmd (Hit Enter)
- then paste in the code bellow with the path changed to the file thats locked

In [ ]:
# attrib -r -h -s /s /d "C:\RawDataLocal\20250910\christielab\session001\*"
# takeown /f "C:\RawDataLocal\20250910\christielab\session001" /r /d y
# icacls "C:\RawDataLocal\20250910\christielab\session001" /grant %USERNAME%:F /t
# rmdir /s /q "\\?\C:\RawDataLocal\20250910\christielab\session001"

In [ ]:
import numpy as np
import pandas as pd

file = 'C:\\RawDataLocal\\20250929\\christielab\\session006\\20250929_christielab_session007_frontCam_stim_frames.npy'
data = np.load(file)
data.shape

(0,)